### Dependencies

In [ ]:
import os
import pickle
from dataclasses import dataclass
from pathlib import Path

import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from recommender_system.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from recommender_system.utils import create_directories, read_yaml

seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

os.chdir("../")

### Entity

In [ ]:
@dataclass(frozen=True)
class TrainModelConfig:
    """Represents the configuration for training the model."""
    root_dir: Path
    base_model_path: Path
    trained_model_path: Path
    data_path: Path
    batch_size: int
    epochs: int
    verbose: int

### Configuration Manager

In [ ]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        """Initialises ConfigurationManager with config and params filepaths."""
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_train_model_config(self) -> TrainModelConfig:
        """Returns the model training configuration."""
        base_model_config = self.config.build_model
        train_model_config = self.config.train_model
        data_config = self.config.data_preprocessing
        create_directories([train_model_config.root_dir])

        train_model_config = TrainModelConfig(
            root_dir=Path(train_model_config.root_dir),
            base_model_path=Path(base_model_config.model_path),
            trained_model_path=Path(train_model_config.trained_model_path),
            data_path=Path(data_config.root_dir),
            batch_size=self.params.BATCH_SIZE,
            epochs=self.params.EPOCHS,
            verbose=self.params.VERBOSE
        )

        return train_model_config

### Component

In [ ]:
class ModelTrainer:
    def __init__(self, config):
        """Initialises the ModelTrainer object with the given config."""
        self.config = config

    def get_base_model(self):
        """Returns the base model."""
        return tf.keras.models.load_model(self.config.base_model_path)

    def load_data(self, data_file_path):
        """Loads and returns data."""
        with data_file_path.open("rb") as f:
            data = pickle.load(f)

        return data

    def get_data(self):
        """Returns the loaded training and validation data."""
        data_dir = self.config.data_path

        X_train = self.load_data(data_dir / "X_train.pkl")
        X_val = self.load_data(data_dir / "X_val.pkl")
        y_train_scaled = self.load_data(data_dir / "y_train_scaled.pkl")
        y_val_scaled = self.load_data(data_dir / "y_val_scaled.pkl")

        return X_train, X_val, y_train_scaled, y_val_scaled

    def get_checkpoint_best_only(self):
        """Returns the checkpoint callback for saving the best model."""
        checkpoint_path = str(self.config.trained_model_path)

        checkpoints = ModelCheckpoint(
            filepath=checkpoint_path,
            monitor="val_accuracy",
            verbose=self.config.verbose,
            save_best_only=True,
            save_weights_only=False,
            save_freq="epoch"
        )

        return checkpoints

    def get_early_stopping(self):
        """Returns the early stopping callback."""
        callback = EarlyStopping(
            patience=2, monitor="val_loss", verbose=self.config.verbose
        )

        return callback

    def initialise_callbacks(self):
        """Initialises and returns the list of callbacks."""
        checkpoint_best_only = self.get_checkpoint_best_only()
        early_stopping = self.get_early_stopping()
        callbacks = [checkpoint_best_only, early_stopping]

        return callbacks

    def train_model(self):
        """Trains the model and returns the training history."""
        X_train, X_val, y_train_scaled, y_val_scaled = self.get_data()
        model = self.get_base_model()
        callbacks = self.initialise_callbacks()

        history = model.fit(
            X_train,
            y_train_scaled,
            batch_size=self.config.batch_size,
            epochs=self.config.epochs,
            verbose=self.config.verbose,
            callbacks=callbacks,
            validation_data=(X_val, y_val_scaled)
        )

        return history

### Pipeline

In [ ]:
try:
    config = ConfigurationManager()
    train_config = config.get_train_model_config()
    model_trainer = ModelTrainer(train_config)
    history = model_trainer.train_model()
except Exception as e:
    raise e